<a href="https://colab.research.google.com/github/andrewcgaitskell/dmtoolnotes/blob/main/SeparateDropDown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
%%sh
pip install -q ipywidgets
pip install -q dash
pip install -q dash_core_components
pip install -q dash_bootstrap_components
pip install -q dash_table
pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [110]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from dash import Dash, Input, Output, callback, State

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
##COMPONENT_STYLE = "/assets/my_component.css"
COMPONENT_STYLE = "/assets/table.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

/usr/local/lib/python3.7/dist-packages/jupyter_dash/jupyter_app.py:98: UserWarning:

The requests_pathname_prefix argument is ignored when running in Colab

/usr/local/lib/python3.7/dist-packages/jupyter_dash/jupyter_app.py:98: UserWarning:

The routes_pathname_prefix argument is ignored when running in Colab



In [111]:
import pandas as pd

In [112]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')
df = df[['continent', 'country', 'pop', 'lifeExp']]  # prune columns for example
df

,continent,country,pop,lifeExp
0,Asia,Afghanistan,31889923.0,43.828
1,Europe,Albania,3600523.0,76.423
2,Africa,Algeria,33333216.0,72.301
3,Africa,Angola,12420476.0,42.731
4,Americas,Argentina,40301927.0,75.320
...,...,...,...,...
137,Asia,Vietnam,85262356.0,74.249
138,Asia,West Bank and Gaza,4018332.0,73.422
139,Asia,"Yemen, Rep.",22211743.0,62.698
140,Africa,Zambia,11746035.0,42.384


In [113]:
df['Mock Date'] = [
    datetime.datetime(2020, 1, 1, 0, 0, 0) + i * datetime.timedelta(hours=13)
    for i in range(len(df))]

In [114]:

def get_band(lifeExp):
    band_raw = 10 * int(lifeExp/10)
    return band_raw

In [115]:
df['band'] = df.apply(lambda x: get_band(x['lifeExp']), axis=1)
band_list = df['band'].unique().tolist()
band_list

[40, 70, 80, 60, 50, 30]

In [116]:
continent_list = df['continent'].unique().tolist()
continent_df = pd.DataFrame(data=continent_list, columns=['Continent'])
#continent_list
continent_df

,Continent
0,Asia
1,Europe
2,Africa
3,Americas
4,Oceania


In [117]:
table = dash_table.DataTable(
    id='datatable1',
    columns=[
        {'name': 'Continent', 'id': 'continent', 'type': 'numeric'},
        {'name': 'Country', 'id': 'country', 'type': 'text'},
        {'name': 'Population', 'id': 'pop', 'type': 'numeric'},
        {'name': 'Life Expectancy', 'id': 'lifeExp', 'type': 'numeric'},
        {'name': 'Mock Dates', 'id': 'Mock Date', 'type': 'datetime'},
        {'name': 'band', 'id': 'band', 'type': 'numeric'}
    ],
    data=df.to_dict('records'),
    filter_action='native',

    style_table={
        'height': 400,
    },
    style_data={
        'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',
    }
)


In [118]:
continent_table = dash_table.DataTable(
    id='continent_table',
    columns=[
        {'name': 'Continent', 'id': 'continent', 'type': 'text'},
    ],
    data=continent_df.to_dict('records'),
    filter_action='native',

    style_table={
        'height': 150,
    },
    style_data={
        'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
        ##'overflow': 'hidden',
        ##'textOverflow': 'ellipsis',
    }
)


In [119]:
continent_dropdown_div = html.Div([
    dcc.Dropdown(continent_list, 'Europe', id='continent-dropdown')
])

lifeexp_dropdown_div = html.Div([
    dcc.Dropdown(band_list, '70', id='lifeexp-dropdown')
])

In [120]:
app.layout = html.Div([continent_dropdown_div,lifeexp_dropdown_div, continent_table, table])

@app.callback(
    output=Output('datatable1', "data"),
    inputs=[Input('continent-dropdown', 'value'), Input('lifeexp-dropdown', 'value')]
    )
def update_graph(continent,lifeband):

    if continent is None:
      dff1 = df;
    else:
      dff1 = df[df['continent']==continent]

    if lifeband is None:
      dff1 = dff1;
    else:
      dff1 = dff1[dff1['band']==lifeband]

    data1=dff1.to_dict("records")

    return data1


In [121]:
app.run_server(mode='inline', debug=True)

<IPython.core.display.Javascript object>